In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
from matplotlib import pyplot as plt
import seaborn as sns
rc('font', family = 'AppleGothic')

In [2]:
train = pd.read_csv("EDA/data/train.csv")
data = pd.read_csv('external_data.csv',header='infer')

## 0. 전처리

- 세대수 = 총세대수
- 기본임대보증금 = 임대보증금
- 주택유형명 = 임대건물구분
- 공급유형명 = 공급유형
- 광역도시명 = 지역
- 공급전용면적 = 전용면적
- 주차수 = 단지내주차면수(완전히 동일하지는 않음!!)

In [3]:
train['임대료'] = train['임대료'].replace('-',np.nan)
train['임대보증금'] = train['임대보증금'].replace('-',np.nan)
train['임대료'] = train['임대료'].apply(lambda x : float(x))
train['임대보증금'] = train['임대보증금'].apply(lambda x : float(x))

In [4]:
data = data.rename(columns = {'세대수':'총세대수','기본임대보증금':'임대보증금','주택유형명':'임대건물구분','공급유형명':'공급유형','광역시도명':'지역','공급전용면적':'전용면적','주차수':'단지내주차면수'})

In [5]:
print(sorted(train['지역'].unique()))
print(sorted(data['지역'].unique()))

['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']
['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']


In [6]:
print(sorted(data['공급유형'].unique()))
print(sorted(train['공급유형'].unique()))

['10년임대', '50년임대', '5년임대', '국민임대', '매입임대', '영구임대', '장기전세', '행복주택']
['공공분양', '공공임대(10년)', '공공임대(50년)', '공공임대(5년)', '공공임대(분납)', '국민임대', '영구임대', '임대상가', '장기전세', '행복주택']


In [7]:
data.loc[data.공급유형.isin(['10년임대']), '공급유형'] = '공공임대(10년)'
data.loc[data.공급유형.isin(['50년임대']), '공급유형'] = '공공임대(50년)'
data.loc[data.공급유형.isin(['5년임대']), '공급유형'] = '공공임대(5년)'

In [8]:
data1 = data[data['임대건물구분']=='아파트']
train1 = train[train['임대건물구분']=='아파트']

In [9]:
data1 = data1[['지역','총세대수','전용면적','단지명','공급유형','임대건물구분','임대보증금','단지내주차면수','기관명','도로명주소']]
train1 = train1[['단지코드','총세대수','지역','전용면적','임대보증금','공급유형','임대건물구분','임대보증금','단지내주차면수']]

## 기준1 : 지역, 공급유형, 총세대수

In [10]:
result = pd.merge(train1,data1,how='left',on=['지역','공급유형','총세대수'])[['단지코드','단지명']].drop_duplicates()

In [11]:
print('NA 개수: ',result['단지명'].isna().sum())
print('특정된 단지코드 수: ',train['단지코드'].nunique()-result['단지명'].isna().sum())
print('전체 단지코드 수: ',train['단지코드'].nunique())
print('NA 비율: ', round(result['단지명'].isna().sum()/train['단지코드'].nunique(),2))

NA 개수:  94
특정된 단지코드 수:  329
전체 단지코드 수:  423
NA 비율:  0.22


In [12]:
result.head()

,단지코드,단지명
0,C2483,포항장량휴먼시아1단지
32,C2515,미수휴먼시아
60,C1407,도안 휴먼시아 4단지
137,C1945,호평마을 주공휴먼시아
155,C1470,장신휴먼시아1단지


## 기준 2 : 지역, 공급유형, 전용면적, 임대보증금

In [13]:
datana = result[result['단지명'].isna()==True]['단지코드'].values
n = len(datana)

In [14]:
datana = train.loc[train['단지코드'].isin(datana)]
datana = datana[datana['임대건물구분']=='아파트']

In [15]:
result2 = pd.merge(datana,data1,how='left',on=['지역','공급유형','전용면적','임대보증금'])[['단지코드','단지명']].drop_duplicates()

In [16]:
result2.head()

,단지코드,단지명
0,C1925,입암주공3단지아파트
2,C1874,쌍용주공7단지1차아파트
4,C1874,공주옥룡주공1차아파트
6,C1874,읍내동 주공아파트
17,C2416,진주평거 주공2단지


In [17]:
print('NA 개수: ',result2['단지명'].isna().sum())
print('특정된 단지코드 수: ',n-result2['단지명'].isna().sum())
print('전체 단지코드 수: ',n)
print('NA 비율: ', round(result2['단지명'].isna().sum()/train['단지코드'].nunique(),2))

NA 개수:  39
특정된 단지코드 수:  55
전체 단지코드 수:  94
NA 비율:  0.09


## 기준3 : 지역, 공급유형, 전용면적, 단지내주차면수

In [18]:
datana = result2[result2['단지명'].isna()==True]['단지코드'].values
n = len(datana)

In [19]:
datana = train.loc[train['단지코드'].isin(datana)]
datana = datana[datana['임대건물구분']=='아파트']

In [20]:
result3 = pd.merge(datana,data1,how='left',on=['지역','공급유형','전용면적','단지내주차면수'])[['단지코드','단지명']].drop_duplicates()

In [21]:
print('NA 개수: ',result3['단지명'].isna().sum())
print('특정된 단지코드 수: ',n-result3['단지명'].isna().sum())
print('전체 단지코드 수: ',n)
print('NA 비율: ', round(result3['단지명'].isna().sum()/train['단지코드'].nunique(),2))

NA 개수:  28
특정된 단지코드 수:  11
전체 단지코드 수:  39
NA 비율:  0.07


- 최종 NA : 28

- 아래 데이터는 하나의 단지코드 별로 두개 이상의 단지명 보유 $\rightarrow$ 잘못 merge된 케이스

In [22]:
result = result.dropna()
result = result.append(result2.dropna())
result= result.append(result3.dropna()).reset_index(drop=True)

In [23]:
tmp = pd.DataFrame(result.groupby('단지코드').count()['단지명']).reset_index()
tmp = np.array(tmp[tmp['단지명']>=2]['단지코드'])
tmp

array(['C1022', 'C1039', 'C1068', 'C1085', 'C1109', 'C1175', 'C1206',
       'C1207', 'C1234', 'C1258', 'C1269', 'C1307', 'C1341', 'C1344',
       'C1439', 'C1451', 'C1601', 'C1616', 'C1681', 'C1732', 'C1740',
       'C1744', 'C1790', 'C1859', 'C1874', 'C1875', 'C1894', 'C1965',
       'C1970', 'C2034', 'C2038', 'C2070', 'C2082', 'C2085', 'C2109',
       'C2127', 'C2132', 'C2133', 'C2135', 'C2173', 'C2186', 'C2190',
       'C2227', 'C2232', 'C2258', 'C2289', 'C2310', 'C2325', 'C2352',
       'C2378', 'C2393', 'C2416', 'C2453', 'C2530', 'C2563', 'C2597',
       'C2621', 'C2627', 'C2635', 'C2657'], dtype=object)

In [24]:
drop = [460, 462, 465, 469, 470, 411, 414, 415, 416, 417, 418, 419, 420, 421, 423, 424 ,425, 426, 427, 428,432,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,450,452,453,454,456,458,459,363, 365,366,370,371,372,373,375,378,379,380,381,382,385,386,388,389,390,391,393,394,395,396,391,393,394,395,396,406]

In [25]:
result = result.drop(drop,axis=0).reset_index(drop=True)

In [26]:
train = pd.merge(train,result,how='left',on='단지코드')

In [27]:
data1 = data1[['단지명','기관명','도로명주소']]

In [28]:
train = pd.merge(train,data1,how='left',on=['단지명'])

In [29]:
store = train[train['임대건물구분']=='상가'][['단지코드','지역','단지명','도로명주소','전용면적']].drop_duplicates().reset_index(drop=True)
store['임대료'] = 0
store['임대보증금'] = 0

In [30]:
store.to_csv("store.csv",index=False)

In [31]:
result.to_csv("result.csv",index=False)

In [32]:
store['단지코드'].unique()

array(['C1925', 'C1874', 'C2416', 'C2621', 'C1616', 'C1704', 'C2258',
       'C2038', 'C1859', 'C1722', 'C2190', 'C1476', 'C1983', 'C2135',
       'C2034', 'C1109', 'C2289', 'C2597', 'C2310', 'C2132', 'C1439',
       'C1899', 'C1056', 'C2644', 'C1206', 'C1775', 'C1790', 'C2109',
       'C1698', 'C1004', 'C1875', 'C2212', 'C2571'], dtype=object)